# 데이터 로딩

In [1]:
import pandas as pd
#raw data loading
df_it = pd.read_csv('Article_IT과학_201906_201908.csv',encoding='euc-kr',names=['날짜','분야','신문사','제목','본문','URL'])
df_politic = pd.read_csv('Article_정치_201906_201908.csv',encoding='euc-kr',names=['날짜','분야','신문사','제목','본문','URL'])
df_economy = pd.read_csv('Article_경제_201906_201908.csv',encoding='euc-kr',names=['날짜','분야','신문사','제목','본문','URL'])
df_life = pd.read_csv('Article_생활문화_201906_201908.csv',encoding='euc-kr',names=['날짜','분야','신문사','제목','본문','URL'])

In [ ]:
# 중복기사 제거
df_it = df_it.drop_duplicates(['신문사', '본문'], keep='first')
df_politic = df_politic.drop_duplicates(['신문사', '본문'], keep='first')
df_economy = df_economy.drop_duplicates(['신문사', '본문'], keep='first')
df_life = df_life.drop_duplicates(['신문사', '본문'], keep='first')

# 데이터 전처리

### 기사 내용에서 명사추출

### 추출한 명사로 Dictionary 생성

### 원핫인코딩 or 임베딩 처리

In [ ]:
df_it['본문'][0]

In [ ]:
from konlpy.tag import Okt
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from multiprocessing import Process, Queue
import getN

In [ ]:
data = [df_it['본문'], df_politic['본문'], df_economy['본문'], df_life['본문']]

In [ ]:
# 4개 category 한번에 명사추출하려고 multiprocessing 한것
if __name__ == '__main__':
    q = Queue()    
    for cate in data:
        #sample 수는 알아서 조절
        sample = cate[0:10000]
        proc = Process(target=getN.getnouns, args=(sample, q))
        proc.start()      

In [ ]:
# 명사 추출한게 queue에 들어있는데 이걸 result 리스트에 저장
result = []
while q.qsize() != 0:
    result.append(q.get())

In [ ]:
# 원래 data의 첫번째 기사 읽고 result의 [0]번째 결과랑 확인 해보면서 category 찾으세요
data[0][0]

In [ ]:
data[1][0]

In [ ]:
data[2][0]

In [ ]:
data[3][0]

In [ ]:
#Tokenizer
import json
#json파일 읽어서 tokenizer로 사용
f = open('5000.json','r')
jsondata = f.read()
all = json.loads(jsondata)
# 5000개도 해보고 10000개도 해보고
tokenizer_5 =Tokenizer(num_words=5000) 

In [ ]:
tokenizer_5.word_index = json.loads(all["config"]["word_index"])
tokenizer_5.index_word = json.loads(all["config"]["index_word"])
tokenizer_5.word_counts = json.loads(all["config"]["word_counts"])

In [ ]:
# result 리스트 결과 ex) 0: 경제 , 1: 정치 2: 생활 3: IT
ec_result = tokenizer_5.texts_to_matrix(result[0], mode='binary')
po_result = tokenizer_5.texts_to_matrix(result[1], mode='binary')
li_result = tokenizer_5.texts_to_matrix(result[2], mode='binary')
it_result = tokenizer_5.texts_to_matrix(result[3], mode='binary')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
sourcedata = [[0],[1],[2],[3]]
enc.fit(sourcedata)

In [ ]:
import numpy as np
ec_y = np.zeros((10000,1))
po_y = np.zeros((10000,1)) + 1
li_y = np.zeros((10000,1)) + 2
it_y = np.zeros((10000,1)) + 3

In [ ]:
total = np.concatenate((ec_result,po_result,li_result,it_result), axis=0)
y_total = np.concatenate((ec_y,po_y,li_y,it_y), axis=0)
total = np.concatenate((total, y_total), axis=1)

In [ ]:
from sklearn.model_selection import train_test_split 
data_train, data_test, label_train, label_test = train_test_split(total[:,:-1],total[:,-1])
label_train = enc.transform(label_train.reshape(30000,1)).toarray()

In [ ]:
from keras import models, layers
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(5000,)))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(data_train, label_train, epochs=20, batch_size=512, validation_split=0.2, shuffle=True)
history_dict = history.history
history_dict.keys()

import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['accuracy']#TODO
val_acc = history.history['val_accuracy']#TODO
loss = history.history['loss']#TODO
val_loss = history.history['val_loss']#TODO

epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')# ‘bo’는 파란색 점
plt.plot(epochs, val_loss, 'b', label='Validation loss')# ‘b’는 파란색 실선
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


plt.clf()   # 그래프를 초기화
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()